In [1]:
# Benders decomposition for warehouse location problem

import gurobipy as gp
from gurobipy import *
import numpy as np

locations = 3
customers = 5

fixcost = [2, 3, 3]

cost = [[2,3,4,5,7],[4,3,1,2,6],[5,4,2,1,3]]

# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        v_y = model.cbGetSolution(model._y)
        print(v_y)
        LB = model.cbGetSolution(model._z)
        bsp = gp.Model("Subproblem") 
        x = {}
        for i in range(locations):
          for j in range(customers):
             x[i,j] = bsp.addVar(obj=cost[i][j])
        for j in range(customers):
           bsp.addConstr(sum(x[i,j] for i in range(locations)) >= 1)
        for i in range(locations):
          for j in range(customers):
            bsp.addConstr(x[i,j] <= v_y[i])
        bsp.optimize()
        if bsp.objVal > LB:
          v = np.zeros(customers)
          w = np.zeros((locations,customers))
          cm = np.zeros(locations)
          i=0
          for c in bsp.getConstrs():
            if i < customers:
               v[i] = c.getAttr("Pi")
            else: 
              lo = (i-customers) // customers
              cu = (i-customers) % customers
              w[lo,cu] = c.getAttr("Pi")
            i=i+1
          for i in range(locations):
            cm[i] = fixcost[i]+sum(w[i,j] for j in range(customers))
          print(cm)
          print(v)
          model.cbLazy(model._z >= sum(v[j] for j in range(customers))+sum(cm[i]*model._y[i] for i in range(locations)))
        global sb
        sb = bsp
        #bsp.dispose
                       
# Masterproblem
                     
m = gp.Model("Benders Masterproblem")
sb = gp.Model('Subproblem')
y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    y[i] = m.addVar(vtype=GRB.BINARY)
constr1 = {}
constr1[0] = m.addConstr(sum(y[i] for i in range(locations)) >= 1)
m.Params.lazyConstraints = 1
m._y = y
m._z = z
m.optimize(subproblem)

print(m.objVal)
print(m._y)



Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1 rows, 4 columns and 3 nonzeros
Model fingerprint: 0xcdf6ae13
Variable types: 1 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
{0: 1.0, 1: 1.0, 2: 1.0}
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 20 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x6a7331c7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 16 rows and 3 columns
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 12 nonzeros

Iteration    Objective       Primal In

In [53]:
help(GRB.attr)



  Attributes are used throughout the Gurobi interface to query and
  modify model properties.  You refer to them as you would any
  other object attribute.  For example, "print model.numConstrs"
  prints the number of constraints in a model.  You can assign new values to
  some attributes (e.g., model.ModelName = "New"), while others can only
  be queried.  Note that attribute modification is handled in a lazy fashion,
  so you won't see the effect of a change until after the next call to
  Model.update() or Model.optimize().

  Capitalization is ignored in Gurobi attribute names, so
  model.numConstrs and model.NumConstrs are equivalent.

  Gurobi attributes can be grouped into the following categories:

  General model attributes (e.g., model.numConstrs):

    numConstrs: Number of constraints
    numVars: Number of variables
    numSOS: Number of SOS constraints
    numQConstrs: Number of quadrtic constraints
    numGenConstrs: Number of general constraints
    numNZs: Number of no